In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
import sys

sys.path.append('../scripts')
sys.path.append('../')

from scripts import graph_osm_loader
from scripts import pipeline, centroids_graph_builder, clustering
from scripts.path_findings import excraction_pfa, dijkstra_pfa
from scripts import utils
import scripts

In [3]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

723 1046


In [4]:
points = utils.read_points(GRAPH_ID, g, num=1000)

In [5]:
cms_resolver = clustering.LouvainCommunityResolver(resolution=25)
cg = centroids_graph_builder.CentroidGraphBuilder().build(g, cms_resolver)

baseline = dijkstra_pfa.Dijkstra(g)

test_alg = excraction_pfa.ExtractionPfa(
    g = g,
    cg=cg,
    upper=dijkstra_pfa.Dijkstra(cg.g),
    down=dijkstra_pfa.Dijkstra(g)
)

find centroids:   0%|          | 0/143 [00:00<?, ?it/s]

find edges:   0%|          | 0/143 [00:00<?, ?it/s]

In [8]:
stat = pipeline.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=10
).compare()

start 0


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 2


...

find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 8


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 9


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
stat


        err_max:    180.7498
        err_min:    0.0000
        err_median: 5.8505
        err_mean:   9.0262
        
        acceleration_max:    8.3152
        acceleration_min:    0.4268
        acceleration_median: 2.4001
        acceleration_mean:   2.5081
        